In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.5.3
2.8.4


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(6)

In [5]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfail'], pff=False)
CategoricalParameter('A.3_Brate', [0, 1, 2])
RealParameter('A.4_Bmax', 30, 350, resolution=N

In [6]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable_name=['3_RfR 0'], pff=False)
IntegerParameter('4_RfR 0', 0, 1, resolution=None, default=None, variable_name=['4_RfR 0'], pff=False)
IntegerParameter('EWS_DaysToThreat', 0, 4, resolution=None, default=None, variable_name=['EWS_DaysToThreat'], pff=False)
IntegerParameter('A.1_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.1_DikeIncrease 0'], pff=False)
IntegerParameter('A.2_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.2_DikeIncrease 0'], pff=False)
IntegerParameter('A.3_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.3_DikeIncrease 0'

In [7]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}

policies = [
    Policy(
        "Baseline",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 0,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    )]

max_policies=[
    Policy(
        "DI max",
        **dict(
            get_do_nothing_dict(),
            **{"A.1_DikeIncrease 0": 5,"A.2_DikeIncrease 0": 5,"A.3_DikeIncrease 0": 5,"A.4_DikeIncrease 0": 5, "A.5_DikeIncrease 0": 5}
        )
    ),
        Policy(
        "RfR max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1,"2_RfR 0": 1, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 0,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "EWS max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0, "2_RfR 0": 0, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 0,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':4
               }
        )
    ),
        Policy(
        "All max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1, "2_RfR 0": 1, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 5,"A.2_DikeIncrease 0": 5,"A.3_DikeIncrease 0": 5,"A.4_DikeIncrease 0": 5, "A.5_DikeIncrease 0": 5,
               'EWS_DaysToThreat':4
               }
        )
    )]
    
Di_Ge_policies=[
        Policy(
        "Di-Ge RfR-Ov max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 5,"A.2_DikeIncrease 0": 5,"A.3_DikeIncrease 0": 5,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "DiGe RfROv mod",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 3,"A.2_DikeIncrease 0": 2,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "DiGe RfROv mod&EWS",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 3,"A.2_DikeIncrease 0": 2,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':2
               }
        )
    )]

infrastr_policies=[
        Policy(
        "Comb Green",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "Comb Grey",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 1,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 3,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 4,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "Comb GreenEWS",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':2
               }
        ))]

policy_old = [Policy(
        "Policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "Policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "Policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    )]

DR12_OP_v1_manual= [
        Policy(
        "DR1&2 OP3",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 1, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 2,"A.2_DikeIncrease 0": 3,"A.3_DikeIncrease 0": 10,"A.4_DikeIncrease 0": 4, "A.5_DikeIncrease 0": 6,
               'EWS_DaysToThreat':1
               }
        )
    ),
        Policy(
        "DR1&2 OP1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 2,"A.2_DikeIncrease 0": 4,"A.3_DikeIncrease 0": 5,"A.4_DikeIncrease 0": 9, "A.5_DikeIncrease 0": 6,
               'EWS_DaysToThreat':1
               }
        )
    ),
        Policy(
        "DR1&2 OP0",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 4,"A.3_DikeIncrease 0": 9,"A.4_DikeIncrease 0": 5, "A.5_DikeIncrease 0": 6,
               'EWS_DaysToThreat':1
               }
        )
    ),
        Policy(
        "DR1&2 OP2",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 1, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 3,"A.3_DikeIncrease 0": 5,"A.4_DikeIncrease 0": 5, "A.5_DikeIncrease 0": 6,
               'EWS_DaysToThreat':1
               }
        )
    )]

In [8]:
from funs_policy_eval import create_optimization_policies

DR12_OP_v1=create_optimization_policies("OptimizationResults_DR12_v1.csv","DR12_v1_P",dike_model)

policies=policies+Di_Ge_policies+DR12_OP_v1

In [9]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 100 scenarios * 8 policies * 1 model(s) = 800 experiments
100%|████████████████████████████████████████| 800/800 [00:34<00:00, 23.52it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [10]:
experiments, outcomes = results
print(experiments.keys())
experiments.head()

Index(['A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
       'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
       'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
       'A.5_Brate', 'A.5_pfail', 'discount rate 0', '0_RfR 0', '1_RfR 0',
       '2_RfR 0', '3_RfR 0', '4_RfR 0', 'EWS_DaysToThreat',
       'A.1_DikeIncrease 0', 'A.2_DikeIncrease 0', 'A.3_DikeIncrease 0',
       'A.4_DikeIncrease 0', 'A.5_DikeIncrease 0', 'scenario', 'policy',
       'model'],
      dtype='object')


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,4_RfR 0,EWS_DaysToThreat,A.1_DikeIncrease 0,A.2_DikeIncrease 0,A.3_DikeIncrease 0,A.4_DikeIncrease 0,A.5_DikeIncrease 0,scenario,policy,model
0,112,78.169892,10.0,0.970400,336.019637,1.0,0.878390,149.864577,1.5,0.037415,...,0,0,0,0,0,0,0,0,Baseline,dikesnet
1,17,213.676159,10.0,0.239566,254.104530,10.0,0.553493,242.235449,1.5,0.420367,...,0,0,0,0,0,0,0,1,Baseline,dikesnet
2,4,135.249418,1.0,0.411510,320.118479,10.0,0.076136,343.279330,1.0,0.386900,...,0,0,0,0,0,0,0,2,Baseline,dikesnet
3,88,228.817131,1.5,0.537797,292.733849,1.5,0.214850,220.368989,1.5,0.724985,...,0,0,0,0,0,0,0,3,Baseline,dikesnet
4,25,116.641625,1.0,0.746924,197.086849,10.0,0.306544,60.843035,1.0,0.244528,...,0,0,0,0,0,0,0,4,Baseline,dikesnet


In [11]:
from funs_viz import tidy_results

results_df = tidy_results(results,6,1)
results_df.to_csv("results/test_1.csv")

print(results_df.columns)
results_df.head()

Index(['A.0_ID flood wave shape', 'discount rate 0', 'EWS_DaysToThreat',
       'scenario', 'policy', 'model', 'RfR Total Costs',
       'Expected Evacuation Costs', 'Dike ring', 'Bmax', 'Brate', 'pfail',
       'RfR 0', 'DikeIncrease 0', 'Dike Investment Costs',
       'Expected Annual Damage', 'Expected Number of Deaths'],
      dtype='object')


,A.0_ID flood wave shape,discount rate 0,EWS_DaysToThreat,scenario,policy,model,RfR Total Costs,Expected Evacuation Costs,Dike ring,Bmax,Brate,pfail,RfR 0,DikeIncrease 0,Dike Investment Costs,Expected Annual Damage,Expected Number of Deaths
0,112,3.5,0,0,Baseline,dikesnet,0.0,0.0,A.1,78.169892,10.0,0.970400,0,0,0,0.000000e+00,0.000000
1,17,3.5,0,1,Baseline,dikesnet,0.0,0.0,A.1,213.676159,10.0,0.239566,0,0,0,7.098540e+08,0.511246
2,4,1.5,0,2,Baseline,dikesnet,0.0,0.0,A.1,135.249418,1.0,0.411510,0,0,0,5.866565e+08,0.195716
3,88,3.5,0,3,Baseline,dikesnet,0.0,0.0,A.1,228.817131,1.5,0.537797,0,0,0,3.568883e+07,0.027325
4,25,1.5,0,4,Baseline,dikesnet,0.0,0.0,A.1,116.641625,1.0,0.746924,0,0,0,0.000000e+00,0.000000
